In [ ]:
import pandas as pd 
import numpy as np

import seaserpent as ss
from scipy.sparse import csc_matrix
from scipy.sparse import csc_matrix, diags
from tqdm import tqdm

In [ ]:
# read from info 
info = ss.Table('info', base='main')
infodf = info.to_frame()
# and remove the ones with bad statuses 
infodf = infodf.loc[~infodf.status.isin(['bad_nucleus','tiny','not_a_neuron','duplicate']),:]
infodf

In [ ]:
infodf.super_class.value_counts()

In [ ]:
cb = infodf[infodf.super_class.isin(['central','visual_projection','sensory','ascending','descending','visual_centrifugal','motor','endocrine'])]

In [ ]:
# apparently there are some NA values for newly-added ids 
cb[cb.root_630.isna()]

In [ ]:
cb = cb[~cb.root_630.isna()]
# match the type from conn below 
cb.loc[:,'root_630'] = cb.root_630.astype(np.int64)

In [ ]:
conn = pd.read_feather('/Users/yijieyin/Downloads/syn_proof_analysis_filtered_630.feather')
conn

In [ ]:
# filter for the connectivity between components of the central brain 
conn = conn[conn.pre_pt_root_id.isin(cb.root_630) & conn.post_pt_root_id.isin(cb.root_630)]

In [ ]:
conncb = conn.groupby(['pre_pt_root_id','post_pt_root_id'])['syn_count'].sum().reset_index()
conncb

In [ ]:
len(conncb.pre_pt_root_id.unique())

In [ ]:
pivot_table = conncb.pivot_table(index='pre_pt_root_id', columns='post_pt_root_id', values='syn_count', fill_value=0)
nodes = list(set(conncb['pre_pt_root_id']).union(set(conncb['post_pt_root_id'])))
# Reindex the pivot table to ensure it includes all nodes in both rows and columns
pivot_table = pivot_table.reindex(index=nodes, columns=nodes, fill_value=0)

In [ ]:
inprop = pivot_table.div(pivot_table.sum(axis = 'rows'), axis = 'columns')

In [ ]:
super_classes = dict(zip(cb.root_630, cb.super_class))

selected_indices = [i for i, rid in enumerate(inprop.columns) if super_classes[rid] in ['sensory','ascending']]

In [ ]:
super_classes = dict(zip(cb.root_630, cb.super_class))
inclass = [rid for rid, sclass in super_classes.items() if sclass in ['sensory','ascending']]

In [ ]:
def generate_steps(skids_indices, inprop, step_number, threshold=0): 
    # Ensure inprop is a sparse matrix
    if not isinstance(inprop, csc_matrix):
        inprop = csc_matrix(inprop)
    
    # Create the initial almost-identity matrix
    ini = csc_matrix((inprop.shape[0], inprop.shape[1]))
    diag_values = [1 if i in skids_indices else 0 for i in range(inprop.shape[1])]
    ini.setdiag(diag_values)
    
    steps_fast = []
    for i in tqdm(range(step_number)): 
        if i == 0: 
            unthresholded = ini @ inprop
        else: 
            unthresholded = steps_fast[-1] @ inprop
        
        # Thresholding
        if threshold == 0:
            mask = unthresholded > threshold
        else:
            mask = unthresholded >= threshold
        thresholded = unthresholded.multiply(mask)
        
        # rounding 
        thresholded.data = np.round(thresholded.data, decimals=4) 
        
        # check density 
        density = thresholded.nnz / (thresholded.shape[0] * thresholded.shape[1])
        print(f"Matrix density at step {idx}: {density:.4f}")
        
        steps_fast.append(thresholded)
    
    return steps_fast
